In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import datasets
from sklearn import manifold

%matplotlib inline

In [2]:
data = datasets.fetch_openml(
    'mnist_784',
    version=1,
    return_X_y=True
)
pixel_values, targets = data
targets = targets.astype(int)

In [3]:
print(pixel_values)

       pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0           0       0       0       0       0       0       0       0       0   
1           0       0       0       0       0       0       0       0       0   
2           0       0       0       0       0       0       0       0       0   
3           0       0       0       0       0       0       0       0       0   
4           0       0       0       0       0       0       0       0       0   
...       ...     ...     ...     ...     ...     ...     ...     ...     ...   
69995       0       0       0       0       0       0       0       0       0   
69996       0       0       0       0       0       0       0       0       0   
69997       0       0       0       0       0       0       0       0       0   
69998       0       0       0       0       0       0       0       0       0   
69999       0       0       0       0       0       0       0       0       0   

       pixel10  ...  pixel7

In [1]:
import numpy as np

def simulate_hat_problem(n, trials=10000):
    success_count = 0  # 统计至少有一个人正确的次数
    total_correct = 0  # 统计所有实验中拿对帽子的人数总和

    for _ in range(trials):
        hats = np.arange(n)  # 0, 1, ..., n-1 对应每个人的帽子
        np.random.shuffle(hats)  # 随机打乱
        correct = np.sum(hats == np.arange(n))  # 统计正确匹配的个数

        if correct > 0:
            success_count += 1
        
        total_correct += correct

    prob_at_least_one = success_count / trials
    expected_correct = total_correct / trials

    return prob_at_least_one, expected_correct

# 运行模拟
n = 10  # 例如有10个人
probability, expectation = simulate_hat_problem(n)
print(f"至少一个人拿对帽子的概率: {probability:.4f}")
print(f"期望拿对帽子的人数: {expectation:.4f}")


至少一个人拿对帽子的概率: 0.6331
期望拿对帽子的人数: 1.0071
